# Brief analysis of the US consumer price index (CPI) 

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import signal

In [ ]:
# Get US CPI and fed funds effective rate dfs
df_cpi = pd.read_csv("../data/US_CPI_M.csv")
df_fed = pd.read_csv("../data/FED_FUNDS_EFFECTIVE_RATE_M.csv")

In [ ]:
# Append the CPI estimate for April of 2024 to the CPI df
new_row = pd.DataFrame({"year_month": ["2024-04"], "rate": [3.4]})
df_cpi = pd.concat([df_cpi, new_row], ignore_index=True)

In [ ]:
# Set date as index for both dfs
df_cpi["year_month"] = pd.to_datetime(df_cpi["year_month"])
df_cpi.set_index("year_month", inplace=True)
df_fed["year_month"] = pd.to_datetime(df_fed["year_month"])
df_fed.set_index("year_month", inplace=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

## Datasets basic info

### US CPI

In [ ]:
df_cpi

In [ ]:
df_cpi.describe()

### Federal Reserve funds effective rate

In [ ]:
df_fed

In [ ]:
df_fed.describe()

## CPI long-term analysis

In [ ]:
plt.figure(figsize=(14, 8))

sns.lineplot(data=df_cpi, x=df_cpi.index, y="rate", label="CPI", color="lime", linewidth=0.75)
plt.axhline(y=2, label="Fed 2% target", color="red", linewidth=1)

plt.title("US consumer price index (CPI) over time")
plt.xlabel("Date")
plt.ylabel("Rate (%)")
plt.legend()

plt.show()

In [ ]:
# All time high
df_cpi.loc[df_cpi["rate"].idxmax()]

In [ ]:
# All time low
df_cpi.loc[df_cpi["rate"].idxmin()]

In [ ]:
# Highest 6 peaks
peaks, _ = signal.find_peaks(df_cpi["rate"], distance=36)
df_cpi.iloc[peaks].nlargest(6, "rate").sort_values("year_month").transpose()

In [ ]:
# Lowest 6 valleys
valleys, _ = signal.find_peaks(-df_cpi["rate"], distance=36)
df_cpi.iloc[valleys].nsmallest(6, "rate").sort_values("year_month").transpose()

In [ ]:
# Deflationary periods table (make into the periods and respective rate avg)
df_cpi_def = df_cpi.loc[df_cpi["rate"] < 0]
df_cpi_def["period"] = (df_cpi_def.index.to_series().diff().dt.days > 31).cumsum()
# Aggregate to get begin date, end date, and average rate

In [ ]:
# Get average inflation rate of each decade
df_cpi_dec = df_cpi.groupby((df_cpi.index.year // 10) * 10)
df_cpi_dec = df_cpi_dec["rate"].mean().round(2).reset_index()
df_cpi_dec.columns = ["decade", "average cpi rate"]
df_cpi_dec.set_index("decade").transpose()

In [ ]:
# Get longest period in which rate was closest to the 2% target with 2 percentage points threshold
df_cpi_close_2 = df_cpi[df_cpi["rate"].sub(2).abs().lt(2)]
df_cpi_close_2

- inflation started rising in the second half of 60s, up until 80 all time high. then quickly fell.
- only during the first hald of the 60s inflation was below the 2% target (considered a ceiling in the past).
- also during the 2010s until the end of the pandemic lockdowns inflation was sort of contained below that.
- most of the time, especially during the 70s and early 80s, inflation rate was really high.
- deflation only on 2009 and 2015.

## CPI + fed rates since 2020 analysis

In [ ]:
# Get dfs since 2020
df_cpi_2020 = df_cpi["2020":]
df_fed_2020 = df_fed["2020":]

In [ ]:
plt.figure(figsize=(14, 8))

sns.lineplot(data=df_cpi_2020, x=df_cpi_2020.index, y="rate", label="CPI", color="lime", linewidth=1)
sns.lineplot(data=df_fed_2020, x=df_fed_2020.index, y="rate", label="Fed funds effective rate", color="violet", linewidth=1)
plt.axhline(y=2, label="Fed 2% target", color="red", linewidth=1)

plt.title("US consumer price index (CPI) and Federal Reserve funds effective rate since 2020")
plt.xlabel("Date")
plt.ylabel("Rate (%)")
plt.legend()

plt.show()

In [ ]:
# when did inflation jumped
df_cpi["2021-01-01":"2021-05-01"]

In [ ]:
# Highest inflation rate
df_cpi_2020.loc[df_cpi_2020["rate"].idxmax()]

In [ ]:
# when inflation became sticky
# df_cpi["2023-04-01":"2023-07-01"]

In [ ]:
# avg inflation since
df_cpi["2023-06-01":].mean()

In [ ]:
# get rate change percentage (when did they really change it?)
# df_fed["2022-01-01":"2022-07-01"]

In [ ]:
# Highest fed funds rate
df_fed_2020.loc[df_fed_2020["rate"].idxmax()]

In [ ]:
df_fed["2023-06-01":].mean() - df_cpi["2023-06-01":].mean()

- As we can see CPI went above 2% on march of 2021, and the next month jumped to 4.16, peaking at about 9% on june of 2022
- the fed took about a year to start rising the rates, when inflation was no longer perceived as transitory.
- rates increased very slowly at first on march of 22, and on june started rising fast, until it peaked on august of 2023, staynin the same since.
- inflation came down until june of 2023, where it stabilized at about 3,3% (still above 2% mark) and became "sticky" since. 
- about 2 percentage points on average the diff